<a href="https://colab.research.google.com/github/sidrusiya/AI-Maze/blob/main/AILA_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AILA Task 1 using NLP

Shared Task by AILA

In [72]:
import numpy as np
import pandas as pd
import os
import nltk
import re
import seaborn as sns
import gensim
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [73]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from google.colab import drive
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
!pip3 install tensorflow_text
import tensorflow_text
!pip install -q tf-models-official
!pip install -q -U tensorflow-text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import tensorflow_hub as hub
import tensorflow_text as tf_text

In [74]:
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/Task1/'
list_dir = os.listdir(PATH)
text = []
labels = []
for file in list_dir:
    file_path = PATH + file
    with open(file_path, 'r') as f:
        for line in f:
            line = line.split(sep='\t')
            text.append(line[0])
            labels.append(line[1][:-1])

data = {'Text': text, "labels": labels}
dataset = pd.DataFrame(data=data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
dataset

,Text,labels
0,The appellant who was arraigned as Accused 2 w...,Facts
1,"The allegations against the accused, inter ali...",Facts
2,Bindula Devi was married to Accused 1 Jaipraks...,Facts
3,The appellant and Accused 3 Shakun Devo Yadav ...,Facts
4,Accused 4 Dani Dutta Yadav is their father and...,Facts
...,...,...
11280,"Suffice it to state that the appellant,as a fa...",Ratio of the decision
11281,"The High Court,in our view,declined,for good r...",Ratio of the decision
11282,We are also not persuaded to take a different ...,Ratio of the decision
11283,"In these circumstances,we are of the view that...",Ratio of the decision


In [76]:
dataset.head(5)

,Text,labels
0,The appellant who was arraigned as Accused 2 w...,Facts
1,"The allegations against the accused, inter ali...",Facts
2,Bindula Devi was married to Accused 1 Jaipraks...,Facts
3,The appellant and Accused 3 Shakun Devo Yadav ...,Facts
4,Accused 4 Dani Dutta Yadav is their father and...,Facts


In [77]:
dataset.shape

(11285, 2)

In [78]:
dataset.labels.value_counts()

Ratio of the decision      4211
Facts                      2622
Precedent                  1787
Argument                    939
Statute                     902
Ruling by Lower Court       483
Ruling by Present Court     341
Name: labels, dtype: int64

# Removing the punctuations,numbers and special characters

In [79]:
for i in dataset.index:
    dataset['Text'][i] = re.sub('[^a-zA-Z0-9 ]','',dataset['Text'][i])
    
dataset.head(10)

,Text,labels
0,The appellant who was arraigned as Accused 2 w...,Facts
1,The allegations against the accused inter alia...,Facts
2,Bindula Devi was married to Accused 1 Jaipraks...,Facts
3,The appellant and Accused 3 Shakun Devo Yadav ...,Facts
4,Accused 4 Dani Dutta Yadav is their father and...,Facts
5,Accused 6 Fudai Yadav is brotherinlaw of Accus...,Facts
6,The prosecution story is reflected in the evid...,Facts
7,He stated that his daughter Bindula Devi was m...,Facts
8,He further stated that in the marriage one buf...,Facts
9,However the accused were not satisfied with that,Facts


# Lowering the characters and removing the stopwords

In [80]:
text = []
for i in dataset.index:
    dataset['Text'][i] = re.sub('[^a-zA-Z0-9 ]','',dataset['Text'][i])
    temp = dataset['Text'][i].lower().split()
    clean = [word for word in temp if word not in stopwords.words('english')]
    clean = " ".join(clean)
    text.append(clean)

In [81]:
text[0]

'appellant arraigned accused 2 tried along five accused offences punishable sections 498a 302 read ss 149 201 ipc 1st additional sessions judge madhepura'

In [82]:
type(text)

list

# Tokenization

In [83]:
for i in range(len(text)):
    text[i] = text[i].split()

In [84]:
text[0]

['appellant',
 'arraigned',
 'accused',
 '2',
 'tried',
 'along',
 'five',
 'accused',
 'offences',
 'punishable',
 'sections',
 '498a',
 '302',
 'read',
 'ss',
 '149',
 '201',
 'ipc',
 '1st',
 'additional',
 'sessions',
 'judge',
 'madhepura']

# Stemmation

In [85]:
from nltk import PorterStemmer

In [86]:
st = PorterStemmer()

In [87]:
for i in range(len(text)):
    text[i] = [st.stem(word) for word in text[i]]

# Recombining Tokens

In [88]:
for i in range(len(text)):
    text[i] = " ".join(text[i])

In [89]:
text[0]

'appel arraign accus 2 tri along five accus offenc punish section 498a 302 read ss 149 201 ipc 1st addit session judg madhepura'

# Using Tf-Idf Vectorizer

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [91]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(text)

In [92]:
type(X)

scipy.sparse.csr.csr_matrix

In [93]:
X = X.toarray()

In [94]:
type(X)

numpy.ndarray

In [95]:
X.shape

(11285, 14871)

In [96]:
Y = dataset['labels']
Y

0                          Facts
1                          Facts
2                          Facts
3                          Facts
4                          Facts
                  ...           
11280      Ratio of the decision
11281      Ratio of the decision
11282      Ratio of the decision
11283      Ratio of the decision
11284    Ruling by Present Court
Name: labels, Length: 11285, dtype: object

# Splitting into train & test dataset

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [99]:
X_train.shape

(9028, 14871)

In [100]:
Y_train.shape

(9028,)

# Applying Logistics Regression

In [101]:
from sklearn.linear_model import LogisticRegression
Log_Reg = LogisticRegression(random_state=0,solver='lbfgs')
Log_Reg.fit(X_train,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [102]:
Y_pred=Log_Reg.predict(X_test)

In [103]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
accuracy_score(Y_test,Y_pred)

0.5702259636685866

In [104]:
print(classification_report(Y_test,Y_pred))

                         precision    recall  f1-score   support

               Argument       0.70      0.19      0.30       202
                  Facts       0.58      0.62      0.60       507
              Precedent       0.64      0.44      0.52       349
  Ratio of the decision       0.53      0.80      0.64       845
  Ruling by Lower Court       0.65      0.13      0.21        87
Ruling by Present Court       0.84      0.27      0.40        79
                Statute       0.74      0.40      0.52       188

               accuracy                           0.57      2257
              macro avg       0.67      0.40      0.45      2257
           weighted avg       0.61      0.57      0.54      2257



# Applying Random Forest

In [105]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [106]:
Y_pred=rf.predict(X_test)

In [107]:
accuracy_score(Y_test,Y_pred)

0.5480726628267611

In [108]:
print(classification_report(Y_test,Y_pred))

                         precision    recall  f1-score   support

               Argument       0.68      0.19      0.30       202
                  Facts       0.57      0.56      0.56       507
              Precedent       0.65      0.34      0.44       349
  Ratio of the decision       0.50      0.82      0.62       845
  Ruling by Lower Court       0.83      0.11      0.20        87
Ruling by Present Court       0.80      0.30      0.44        79
                Statute       0.75      0.38      0.50       188

               accuracy                           0.55      2257
              macro avg       0.68      0.39      0.44      2257
           weighted avg       0.60      0.55      0.52      2257



In [109]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [110]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}
